In [1]:
#import libraries
import pandas as pd
import sqlalchemy
import numpy as np

In [2]:
engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost:3306/Lucid')

In [61]:
posts = pd.read_sql_table('posts', engine,index_col='id')
posts.drop(['status_id','action','post_id'], axis = 1, inplace=True)
posts.head(20)

,user_id,title,content,tags,slug,created_at,updated_at,image
id,,,,,,,,
1,2077,What i have learnt so far on HTML,I learnt how to use the table tag as i have us...,None,what-i-have-learnt-so-far-on-html-985,2019-08-25 20:39:01,2019-08-28 12:30:00,1
2,1719,HTML BEGINS HERE,"I am on this journey with start.ng, and here ...",Technology,html-begins-here-4d6,2019-08-25 20:44:36,2019-08-28 12:30:00,1
4,1310,My Laziness In The Open,I have not been attending classes on the HNG c...,None,my-laziness-in-the-open-029,2019-08-25 20:50:33,2019-08-28 12:30:00,1
6,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,None,my-task-2-649,2019-08-25 20:51:36,2019-08-28 12:30:00,1
7,167,Task 2,"A Summary on The “idongesit.html” CV, Its Str...",None,task-2-7e5,2019-08-25 20:51:39,2019-08-28 12:30:00,1
8,329,My Journey on HTML,Using the Hyper Text Markup Language (HTML) ha...,None,my-journey-on-html-ce5,2019-08-25 20:54:07,2019-08-28 12:30:34,1
9,2040,StartNG HTML Exposition,![](/storage/2040/images/img-kf6sy3kvg0.png)![...,None,startng-html-exposition-4d5,2019-08-25 20:55:20,2019-08-28 12:30:00,1
11,1787,MY TASK 2,My journey on **StartNG** pre-internship progr...,html,my-task-2-649,2019-08-25 20:57:59,2019-08-28 12:30:00,1
12,2040,StartNG HTML task,<p> </p>\n<p>I have learned a lot about HTML a...,,startng-html-task-8e2,2019-08-25 21:01:15,2019-08-28 12:30:00,daraoloye99/images/img-gpzb56s7g1.png


In [5]:
#read notifications table
notifications = pd.read_sql_table('notifications',engine)
notifications.dropna(axis = 1,inplace=True)
notifications.drop(['id'], axis = 1, inplace=True)
notifications.head()

,sender_id,user_id,status,action,type,created_at,updated_at
0,4,7,1,Followed,Following,2019-08-26 10:43:56,2019-08-26 19:20:28
1,1,4,1,Followed,Following,2019-08-26 10:48:21,2019-08-26 14:05:42
2,3,45,0,Followed,Following,2019-08-26 10:51:39,2019-08-26 10:51:39
3,1,4,1,Followed,Following,2019-08-26 12:32:46,2019-08-26 14:05:42
4,1,4,1,Commented,Post,2019-08-26 16:13:17,2019-08-26 16:13:25


In [7]:
thoughts = pd.read_sql_table('thoughts',engine,index_col='id')
thoughts.head()

,user_id,content,created_at,updated_at
id,,,,
1,2195,hhhhhhhh,2019-08-26 14:30:26,2019-08-26 14:30:26
2,148,Can't believe I'm learning ML,2019-08-26 14:57:35,2019-08-26 14:57:35
3,2212,- what you have learned so far\r\nI've learned...,2019-08-26 15:35:41,2019-08-26 15:35:41
4,1,Hey people,2019-08-26 16:12:07,2019-08-26 16:12:07
5,2223,nice,2019-08-26 16:24:55,2019-08-26 16:24:55


In [50]:
df2 = pd.merge(posts,notifications,on='user_id')
df2.drop(['tags','created_at_x','updated_at_x','created_at_y','updated_at_y'], axis = 1, inplace = True)
#df2.head(40)

In [51]:
posts['content'].head()

id
1    I learnt how to use the table tag as i have us...
2     I am on this journey with start.ng, and here ...
4    I have not been attending classes on the HNG c...
6    My journey on **StartNG** pre-internship progr...
7     A Summary on The “idongesit.html” CV, Its Str...
Name: content, dtype: object

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
tfidf = TfidfVectorizer(stop_words='english')

In [52]:
posts['content'] = posts['content'].fillna('')

In [53]:
tfidf_matrix = tfidf.fit_transform(posts['content'])

In [54]:
tfidf_matrix.shape

(876, 5122)

In [55]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)

In [57]:
indices = pd.Series(posts.index, index = posts['title']).drop_duplicates()

In [66]:
def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    content_indices = [i[0] for i in sim_scores]
    print('Articles you might be interested in:\n')
    return set(posts['title'].iloc[content_indices])

In [68]:
get_recommendations('Task Number 7')

Articles you might be interested in:



{'First HTML Project',
 'HTML',
 'My First Language!!',
 'The Building Block',
 'What i learnt  so far about HTML',
 'What is HTML',
 'what is HTML'}